In [ ]:
import re
import numpy
import jsonlines

def format_the_string(string, like_onto = False):
    formatted_array = []
    if like_onto:
        regex = re.compile(r"([a-zA-Z0-9]*)|([a-zA-Z0-9]*)([\.?!,])|([a-zA-Z0-9]*)(-)([a-zA-Z0-9]*)|([a-zA-Z0-9]*)('[a-zA-Z0-9]*)")        
    else:    
        regex = re.compile(r"([a-zA-Z0-9]*)|([a-zA-Z0-9]*)([\.?!,])|([a-zA-Z0-9]*)(-)([a-zA-Z0-9]*)|([a-zA-Z0-9]*)('[a-zA-Z0-9]*)|([\[]*)|([\]]*)|([\*]*)")
    for tup in re.findall(regex, string):
        for x in tup:
            if x == "":
                continue
            else:
                formatted_array.append(x)
    return formatted_array


def remove_values_from_array(array, val):
    return [value for value in array if value != val]


def remove_one_instance_of_replacement(arr):
    new_arr = [arr[0]]
    removed = False
    for mention in arr[1:]:
        if numpy.array_equal(mention, arr[0]) and removed == False:
            removed = True 
        else:
            new_arr.append(mention)
    return new_arr

            
def check_speaker(num_of_sentence, speakers):
    if num_of_sentence < len(speakers):
        return speakers[num_of_sentence][0]
    else:
        return speakers[-1][0]
        


def prune_gold_clusters(clusters, last_index):
    pruned_clusters = []
    for cluster in clusters:
        pruned_cluster = []
        for mention in cluster:
            if mention[0] > last_index or mention[1] > last_index:
                continue
            else:
                pruned_cluster.append(mention)
        pruned_clusters.append(pruned_cluster)                
    return pruned_clusters   


def get_all_words_from_sentences(sentences):
    all_words = []
    for sentence in sentences:
        for word in sentence:
            all_words.append(word)
    return all_words


def turn_gold_cluster_indices_to_words(all_words, clusters):
    word_clusters = []
    for cluster in clusters:
        word_cluster = []
        for part in cluster:
            word_part = []
            i = part[0]
            while i <= part[1]:
                word_part.append(all_words[i])
                i = i + 1
            word_cluster.append(word_part)
        word_clusters.append(word_cluster)
    return word_clusters

def isSubArray(A, B, n, m):
    i = 0; j = 0;
    while (i < n and j < m):
        if (A[i] == B[j]):
            i += 1;
            j += 1;
            if (j == m):
                return True;
        else:
            i = i - j + 1;
            j = 0;
    return False;

def find_line(split_og, jsonlines_file):
    our_line = {}
    with jsonlines.open(jsonlines_file) as reader:
            for line in reader:
                line_sentences = [] 
                for sentence in line["sentences"]:
                    line_sentences = line_sentences + sentence
                xs = range(0,len(line_sentences) - 50,50)
                for x in xs:
                    sub_arr = [line_sentences[i] for i in range(x, x+15)]
                    if isSubArray(split_og, sub_arr, len(split_og), 15):
                        our_line = line
                        break                      
    return our_line


text = ""
def lst2str(word_list):
    global text
    special_signs = ["",",",".","!","?",":","'s"]
    for item in word_list:
        if type(item) == list:
            lst2str(item)
        else:
            if len(text) == 0 or item in special_signs:
                text += str(item)
            else:
                text += " " + str(item)
    return text


def remove_mentions_and_added_symbols(arr):
    for i in range(len(arr)):
        if arr[i] == "[[":
            arr[i] = ""
            k = i + 1
            while k < len(arr) and arr[k] != "]]":
                arr[k] = ""
                k += 1
            if k < len(arr):
                arr[k] = ""
                i = k
        if arr[i] == "**":
            arr[i] = ""
    return lst2str(arr)      